В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
import math

import pandas as pd
import scipy
import scipy.stats as stats
import snoop
import statsmodels
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import *

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
experiment, control = 10 * [1] + 24 * [0], 4 * [1] + 12 * [0]

$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [4]:
def proportions_diff_confint_ind(sample1, sample2, alpha=0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.0)

    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)

    left_boundary = (p1 - p2) - z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )
    right_boundary = (p1 - p2) + z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )

    return (left_boundary, right_boundary)

In [5]:
proportions_diff_confint_ind(experiment, control)

(-0.2175577216559601, 0.3057930157736072)

In [6]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)

    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2
    P = float(p1 * n1 + p2 * n2) / (n1 + n2)

    return (p1 - p2) / np.sqrt(P * (1 - P) * (1.0 / n1 + 1.0 / n2))

In [7]:
z = proportions_diff_z_stat_ind(experiment, control)

In [8]:
def proportions_diff_z_test(z_stat, alternative="two-sided"):
    if alternative not in ("two-sided", "less", "greater"):
        raise ValueError(
            "alternative not recognized\n" "should be 'two-sided', 'less' or 'greater'"
        )

    if alternative == "two-sided":
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))

    if alternative == "less":
        return scipy.stats.norm.cdf(z_stat)

    if alternative == "greater":
        return 1 - scipy.stats.norm.cdf(z_stat)

In [ ]:
round(proportions_diff_z_test(z, alternative="greater"), 4)

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции `sklearn.cross_validation.train_test_split `(зафиксируйте `random state = 1`). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам $X_1,X_2,X_3$

логистическая регрессия по признакам $X_4,X_5,X_6$ 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости.

In [42]:
data = pd.read_csv("~/MainDir/С4/Tests/banknotes.txt", sep="\t")
data.to_csv("banknotes.csv")

In [27]:
import bamboolib as bam

In [43]:
data

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [10]:
import pandas as pd

df = pd.read_csv(
    r"/Users/glebsokolov/MainDir/С4/Tests/banknotes.csv", sep=",", decimal="."
)
X, y = df.drop('real', axis=1), df.real

In [123]:
# X_1 = df[["X1", "X2", "X3"]]
# X_2 = df[["X4", "X5", "X6"]]
# y = df[["real"]]
# X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(
#     X_1, y, test_size=50 / 200, random_state=1
# )
# X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(
#     X_2, y, test_size=50 / 200, random_state=1
# )

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [20]:
LogReg1 = LogisticRegression(solver='liblinear')
LogReg1.fit(X_train[["X1", "X2", "X3"]], y_train)
res1 = LogReg1.predict(X_test[["X1", "X2", "X3"]])
arr_1 = [
    res1[i] == np.reshape(y_test.to_numpy(), (-1,))[i] for i in range(len(y_test))
]
# res_1 = sum(
#     [res[i] == np.reshape(y_test.to_numpy(), (-1,))[i] for i in range(len(y_test))]
# ) / len(res)

In [21]:
LogReg2 = LogisticRegression(solver='liblinear')
LogReg2.fit(X_train[["X4", "X5", "X6"]], y_train)
res2 = LogReg2.predict(X_test[["X4", "X5", "X6"]])
arr_2 = [
    res2[i] == np.reshape(y_test.to_numpy(), (-1,))[i] for i in range(len(y_test))
]
# res_2 = sum(
#     [res[i] == np.reshape(y_test.to_numpy(), (-1,))[i] for i in range(len(y_test))]
# ) / len(res)

$$
\begin{array}{|l|l|l|l|}
\hline X_{1} \backslash X_{2} & 1 & 0 & \Sigma \\
\hline 1 & e & f & e+f \\
\hline 0 & g & h & g+h \\
\hline \Sigma & e+g & f+h & h \\
\hline
\end{array}
$$
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [19]:
def proportions_diff_confint_rel(sample1, sample2, alpha=0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.0)
    sample = list(zip(sample1, sample2))
    n = len(sample)

    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])

    left_boundary = float(f - g) / n - z * np.sqrt(
        float((f + g)) / n ** 2 - float((f - g) ** 2) / n ** 3
    )
    right_boundary = float(f - g) / n + z * np.sqrt(
        float((f + g)) / n ** 2 - float((f - g) ** 2) / n ** 3
    )
    return (left_boundary, right_boundary)

In [20]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)

    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])

    return float(f - g) / np.sqrt(f + g - float((f - g) ** 2) / n)

In [26]:
print(
    "95%% confidence interval for a difference between proportions: [%f, %f]"
    % proportions_diff_confint_rel(arr_1, arr_2)
)

95% confidence interval for a difference between proportions: [-0.300055, -0.059945]


In [25]:
print(
    "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(arr_1, arr_2))
)

p-value: 0.003297


In [27]:
np.format_float_scientific(
    proportions_diff_z_test(proportions_diff_z_stat_rel(arr_1, arr_2))
)

'3.2969384555543435e-03'

In [53]:
print(
    "p-value: %f"
    % proportions_diff_z_test(proportions_diff_z_stat_rel(arr_1, arr_2), "less")
)

p-value: 0.000462


In [55]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )
#%%
pvalue = proportions_diff_z_test(proportions_diff_z_stat_rel(arr_1, arr_2))
print ("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(arr_1, arr_2)))

p-value: 0.000923


In [57]:
cinfidence_int_rel = proportions_diff_confint_rel(arr_1, arr_2)
print ("95%% confidence interval for a difference between predictions: [%.4f, %.4f]" \
      % (np.round(cinfidence_int_rel[0],4), np.round(cinfidence_int_rel[1], 4)))

95% confidence interval for a difference between predictions: [-0.2865, -0.0735]


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

$$
\begin{aligned}
Z\left(X_{1}^{n_{1}}, X_{2}^{n_{2}}\right) &=\frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{\frac{\sigma_{1}^{2}}{n_{1}}+\frac{\sigma_{2}^{2}}{n_{2}}}} \\
Z\left(X_{1}^{n_{1}}, X_{2}^{n_{2}}\right) & \sim N(0,1)
\end{aligned}
$$

$$
\nu=\frac{\left(\frac{S_{1}^{2}}{n_{1}}+\frac{S_{2}^{2}}{n_{2}}\right)^{2}}{\frac{S_{1}^{4}}{n_{1}^{2}\left(n_{1}-1\right)}+\frac{S_{2}^{4}}{n_{2}^{2}\left(n_{2}-1\right)}}
$$

$$
\begin{aligned}
&T\left(X_{1}^{n_{1}}, X_{2}^{n_{2}}\right)=\frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{\frac{S_{1}^{2}}{n_{1}}+\frac{S_{2}^{2}}{n_{2}}}} \\
&T\left(X_{1}^{n_{1}}, X_{2}^{n_{2}}\right) \approx \sim S t(\nu) .
\end{aligned}
$$

$$
\begin{aligned}
&Z\left(X^{n}\right)=\frac{\bar{X}-\mu_{0}}{\sigma / \sqrt{n}} \\
&Z\left(X^{n}\right) \sim N(0,1)
\end{aligned}
$$

In [113]:
mean_experiment, mean_control = 541.5, 525
std_experiment, std_control = 100, 100
size_experiment, size_control = 100, 200000 - 100

In [114]:
Z = (mean_experiment - mean_control) / (std_control / math.sqrt(100))

In [115]:
round(1 - stats.norm.cdf(Z), 4)

0.0495